In [1]:
import nltk # KIT DE FERRAMENTAS para Linguagem Natural
nltk.download('stopwords')

from nltk.corpus import stopwords

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython import display
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\T600356\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f # para alguns tratamentos

spark = SparkSession.builder\
    .master('local[*]')\
    .appName('WordCloud')\
    .getOrCreate()

RuntimeError: Java gateway process exited before sending its port number

In [ ]:
def trata_tweets(df):
    words = df\
        .select(f.explode(f.split(f.lower('_c0')," ")).alias('word'))\
        .withColumn('word',f.regexp_replace("word", r'http\S+', ''))\
        .withColumn('word',f.regexp_replace("word", r'@\w+', ''))\
        .withColumn('word',f.regexp_replace("word", r'rt', ''))\
        .na.replace('',None)\
        .na.drop()
    return words


# recebe o DataFrame
# troca as linhas que tem HTTP... por nada
# troca o @ padrão por nada
# troca o rt (retweetando) por nada
# AS LINHAS (por isso que somente replace) que estão com na troca por None
# APAGAMOS A LINHA QUE NÃO TEM NADA NADA.
# depois retornando as palavras do DF depois de tratadas

In [3]:
stops = stopwords.words('portuguese') 
# Passando as palavras TOPS em português
stops.append('futebol')
plt.figure(figsize=(20,10))

while True:
    try:
        words = spark.read.csv('./csv',encoding='utf-8')
        words = trata_tweets(words)
        rows = words.collect() # lista de linhas, para poder gerar o gráfico
        all_words = ''
        for row in rows:
            # para item da lista de linhas
            all_words = all_words + ' ' + row['word']
        
        wordcloud = WordCloud(stopwords=stops, backgorund_color='black',width=1920,height=1080,max_words=100)\
        .generate(all_words)

        # código para gerar nossa WordClound simples
        plt.cla() # limpando os eixo do gráfico
        plt.axis('off') 
        plt.imshow(wordcloud)
        display.display(plt.gcf())
        display.clear_output(wait=True) 
        time.sleep(10)

    except KeyboardInterrupt:
        break # quando der o comando no meu teclado, para encerrar ele encerra.

NameError: name 'spark' is not defined

<Figure size 2000x1000 with 0 Axes>